In [1]:
import quandl
import pandas as pd
import re
import numpy as np

In [2]:
quandl.ApiConfig.api_key = 'es4ABcXZ_usFTxNf4mph'

In [3]:
indicators = ['ZATT','ZSFH','ZABT']
data = quandl.get_table('ZILLOW/DATA', region_id='394913', indicator_id=indicators,paginate=True)
data.date=pd.to_datetime(data.date)
data=data.reset_index()
data=data.drop(['region_id','None'],axis=1)

In [4]:
data

,indicator_id,date,value
0,ZSFH,2023-02-28,588633.772961
1,ZSFH,2023-01-31,588200.022840
2,ZSFH,2022-12-31,588401.051911
3,ZSFH,2022-11-30,601538.479252
4,ZSFH,2022-10-31,623959.000000
5,ZSFH,2022-09-30,624669.000000
6,ZSFH,2022-08-31,625394.000000
7,ZSFH,2022-07-31,624836.000000
8,ZSFH,2022-06-30,621685.000000
9,ZSFH,2022-05-31,617223.000000


In [5]:
pivoted_data = data.pivot_table(index="date", columns=["indicator_id"], values="value")
pivoted_data = pivoted_data.reset_index()

In [6]:
pivoted_data=pivoted_data[(pivoted_data.date>'2015-2-28')]

In [7]:
pivoted_data

indicator_id,date,ZABT,ZATT,ZSFH
230,2015-03-31,256798.000000,7.400920e+05,408852.000000
231,2015-04-30,257328.000000,7.420650e+05,409762.000000
232,2015-05-31,257751.000000,7.436390e+05,410461.000000
233,2015-06-30,258390.000000,7.458940e+05,411435.000000
234,2015-07-31,258905.000000,7.476230e+05,412263.000000
235,2015-08-31,259320.000000,7.492770e+05,413059.000000
236,2015-09-30,259782.000000,7.507640e+05,413902.000000
237,2015-10-31,260256.000000,7.523740e+05,414783.000000
238,2015-11-30,261084.000000,7.549030e+05,416161.000000
239,2015-12-31,261601.000000,7.568260e+05,417239.000000


In [8]:
pivoted_data[['ZABT','ZATT','ZSFH']]=pivoted_data[['ZABT','ZATT','ZSFH']].shift(2)

In [9]:
pivoted_data

indicator_id,date,ZABT,ZATT,ZSFH
230,2015-03-31,NaN,NaN,NaN
231,2015-04-30,NaN,NaN,NaN
232,2015-05-31,256798.000000,7.400920e+05,408852.000000
233,2015-06-30,257328.000000,7.420650e+05,409762.000000
234,2015-07-31,257751.000000,7.436390e+05,410461.000000
235,2015-08-31,258390.000000,7.458940e+05,411435.000000
236,2015-09-30,258905.000000,7.476230e+05,412263.000000
237,2015-10-31,259320.000000,7.492770e+05,413059.000000
238,2015-11-30,259782.000000,7.507640e+05,413902.000000
239,2015-12-31,260256.000000,7.523740e+05,414783.000000


### Pulling From FRED

In [11]:
import pandas_datareader.data as web
import datetime
start=datetime.datetime(2016,1,1)
end=datetime.datetime(2023,2,1)

FRED_REGION=web.DataReader(['NYTPOP','ACTLISCOU35620','NEWLISCOU35620','PENLISCOU35620','PRIREDCOU35620',
                    'NEWY636BPPRIV','SMU36356204200000001','SMU36356200500000001SA','SMU36356204300000001SA',
                    'ENUC356230010SA','NEWY636PCPI','SMU36356200500000003','SMU36356200500000002','SMU36356200500000011',
                    'LAUMT363562000000005','NEWY636URN','NEWY636NA','NEWY636LFN','NEWY636NRMN','NEWY636PBSV',
                    'NEWY636FIRE','NEWY636MFG','NEWY636EDUH','NEWY636LEIH'],'fred',start,end)

In [12]:
FRED_REGION.index =FRED_REGION.index + pd.offsets.MonthEnd(0)
FRED_REGION=FRED_REGION.ffill(axis ='rows')
FRED_REGION=FRED_REGION.reset_index()

In [13]:
FRED_CPI=web.DataReader(['CUURA101SA0','CUURA101SAF11','CUURA101SAF','CUURA101SA0E','CUURA101SEHF01','CUURA101SAT',
'CUURA101SAM','CUURA101SAR','CUURA101SAF116','CUURA101SAA','CUURA101SANL1','CUURA101SS47016'],'fred',start,end)

In [14]:
FRED_CPI.index =FRED_CPI.index + pd.offsets.MonthEnd(0)
FRED_CPI=FRED_CPI.reset_index()

In [15]:
FRED_US=web.DataReader(['PSAVERT','MICH','FEDFUNDS','T10Y2Y','T10Y3M'],'fred',start,end)

In [16]:
FRED_US=FRED_US.ffill(axis ='rows')
FRED_US=FRED_US.resample('M').last()
FRED_US=FRED_US.reset_index()

In [17]:
FRED_US

,DATE,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
0,2016-01-31,7.7,2.5,0.34,1.18,1.61
1,2016-02-29,7.2,2.5,0.38,0.96,1.41
2,2016-03-31,7.6,2.7,0.36,1.05,1.57
3,2016-04-30,7.2,2.8,0.37,1.06,1.61
4,2016-05-31,6.9,2.4,0.37,0.97,1.50
5,2016-06-30,6.6,2.6,0.38,0.91,1.23
6,2016-07-31,6.8,2.7,0.39,0.79,1.18
7,2016-08-31,6.8,2.5,0.40,0.78,1.25
8,2016-09-30,6.8,2.4,0.40,0.83,1.31
9,2016-10-31,6.9,2.4,0.40,0.98,1.50


### Left Joining on Date

In [18]:
RSNA=pd.read_csv('RSNA.csv')
RSNA.date=pd.to_datetime(RSNA.date)

In [19]:
rent=pd.merge(pivoted_data,RSNA,on='date',how='left')

In [20]:
rent

,date,ZABT,ZATT,ZSFH,RSNA
0,2015-03-31,NaN,NaN,NaN,2386.585780
1,2015-04-30,NaN,NaN,NaN,2405.273501
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106


In [21]:
df_region=pd.merge(rent,FRED_REGION,left_on='date',right_on='DATE',how='left')

In [22]:
df_region

,date,ZABT,ZATT,ZSFH,RSNA,DATE,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,...,LAUMT363562000000005,NEWY636URN,NEWY636NA,NEWY636LFN,NEWY636NRMN,NEWY636PBSV,NEWY636FIRE,NEWY636MFG,NEWY636EDUH,NEWY636LEIH
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_region_CPI=pd.merge(df_region,FRED_CPI,on='DATE',how='left')

In [24]:
df_region_CPI

,date,ZABT,ZATT,ZSFH,RSNA,DATE,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,...,CUURA101SAF,CUURA101SA0E,CUURA101SEHF01,CUURA101SAT,CUURA101SAM,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_region_CPI_US=pd.merge(df_region_CPI,FRED_US,on='DATE',how='left')

In [26]:
df_region_CPI_US

,date,ZABT,ZATT,ZSFH,RSNA,DATE,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,...,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_region_CPI_US=df_region_CPI_US.drop('DATE',axis=1)

In [28]:
df_region_CPI_US

,date,ZABT,ZATT,ZSFH,RSNA,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,PRIREDCOU35620,...,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df=df_region_CPI_US.copy()

In [30]:
df

,date,ZABT,ZATT,ZSFH,RSNA,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,PRIREDCOU35620,...,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,256798.000000,7.400920e+05,408852.000000,2422.713710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,257328.000000,7.420650e+05,409762.000000,2440.732457,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257751.000000,7.436390e+05,410461.000000,2449.784467,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,258390.000000,7.458940e+05,411435.000000,2463.399918,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258905.000000,7.476230e+05,412263.000000,2470.269690,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,259320.000000,7.492770e+05,413059.000000,2475.803187,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259782.000000,7.507640e+05,413902.000000,2470.715491,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,260256.000000,7.523740e+05,414783.000000,2461.873106,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
cols_to_shift_1 = df.columns.difference(['date','RSNA','NYTPOP','NEWY636PCPI','SMU36356204200000001', 'SMU36356200500000001SA',
       'SMU36356204300000001SA', 'ENUC356230010SA', 'NEWY636PCPI',
       'SMU36356200500000003', 'SMU36356200500000002', 'SMU36356200500000011',
       'LAUMT363562000000005', 'NEWY636URN', 'NEWY636NA', 'NEWY636LFN',
       'NEWY636NRMN', 'NEWY636PBSV', 'NEWY636FIRE', 'NEWY636MFG',
       'NEWY636EDUH', 'NEWY636LEIH'])
df[cols_to_shift_1] = df[cols_to_shift_1].shift(1)

cols_to_shift_2=['SMU36356204200000001', 'SMU36356200500000001SA',
       'SMU36356204300000001SA', 'ENUC356230010SA', 'NEWY636PCPI',
       'SMU36356200500000003', 'SMU36356200500000002', 'SMU36356200500000011',
       'LAUMT363562000000005', 'NEWY636URN', 'NEWY636NA', 'NEWY636LFN',
       'NEWY636NRMN', 'NEWY636PBSV', 'NEWY636FIRE', 'NEWY636MFG',
       'NEWY636EDUH', 'NEWY636LEIH']

for col in df.columns:
    if col in cols_to_shift_2:
        df[col] = df[col].shift(2)
    else:
        pass      

In [32]:
df

,date,ZABT,ZATT,ZSFH,RSNA,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,PRIREDCOU35620,...,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
0,2015-03-31,NaN,NaN,NaN,2386.585780,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-04-30,NaN,NaN,NaN,2405.273501,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-05-31,NaN,NaN,NaN,2422.713710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-06-30,256798.000000,7.400920e+05,408852.000000,2440.732457,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-31,257328.000000,7.420650e+05,409762.000000,2449.784467,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31,257751.000000,7.436390e+05,410461.000000,2463.399918,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-09-30,258390.000000,7.458940e+05,411435.000000,2470.269690,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-10-31,258905.000000,7.476230e+05,412263.000000,2475.803187,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-11-30,259320.000000,7.492770e+05,413059.000000,2470.715491,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-12-31,259782.000000,7.507640e+05,413902.000000,2461.873106,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df.isna().sum()

date                       0
ZABT                       3
ZATT                       3
ZSFH                       3
RSNA                       2
NYTPOP                    10
ACTLISCOU35620            17
NEWLISCOU35620            17
PENLISCOU35620            17
PRIREDCOU35620            17
NEWY636BPPRIV             11
SMU36356204200000001      12
SMU36356200500000001SA    12
SMU36356204300000001SA    12
ENUC356230010SA           12
NEWY636PCPI               12
SMU36356200500000003      12
SMU36356200500000002      12
SMU36356200500000011      12
LAUMT363562000000005      12
NEWY636URN                12
NEWY636NA                 12
NEWY636LFN                12
NEWY636NRMN               12
NEWY636PBSV               12
NEWY636FIRE               12
NEWY636MFG                12
NEWY636EDUH               12
NEWY636LEIH               12
CUURA101SA0               11
CUURA101SAF11             11
CUURA101SAF               11
CUURA101SA0E              11
CUURA101SEHF01            11
CUURA101SAT   

### Data Filtering

In [34]:
df=df[(df.date>'2016-7-31')&(~df.RSNA.isna())]

In [35]:
df

,date,ZABT,ZATT,ZSFH,RSNA,NYTPOP,ACTLISCOU35620,NEWLISCOU35620,PENLISCOU35620,PRIREDCOU35620,...,CUURA101SAR,CUURA101SAF116,CUURA101SAA,CUURA101SANL1,CUURA101SS47016,PSAVERT,MICH,FEDFUNDS,T10Y2Y,T10Y3M
17,2016-08-31,263665.000000,7.639080e+05,421874.000000,2515.697593,19336.456,71440.0,21816.0,9263.0,12840.0,...,118.985,254.813,121.576,178.913,194.696,6.8,2.7,0.39,0.79,1.18
18,2016-09-30,264317.000000,7.653900e+05,422882.000000,2517.262497,19336.456,69427.0,17092.0,9077.0,11850.0,...,119.552,254.393,129.847,180.747,188.480,6.8,2.5,0.40,0.78,1.25
19,2016-10-31,265031.000000,7.670250e+05,423987.000000,2512.303863,19336.456,67715.0,19382.0,8934.0,12878.0,...,119.874,254.879,133.232,183.620,190.183,6.8,2.4,0.40,0.83,1.31
20,2016-11-30,265862.000000,7.689390e+05,425312.000000,2507.025158,19336.456,66569.0,16736.0,8943.0,11808.0,...,120.142,255.126,130.743,183.239,192.314,6.9,2.4,0.40,0.98,1.50
21,2016-12-31,266632.000000,7.709340e+05,426762.000000,2496.784727,19336.456,62162.0,15042.0,9062.0,9930.0,...,121.092,256.281,124.428,182.221,202.140,7.0,2.4,0.41,1.26,1.89
22,2017-01-31,267494.000000,7.730800e+05,428306.000000,2492.825735,19325.115,55645.0,10668.0,8938.0,5872.0,...,121.369,254.823,119.924,181.856,202.837,6.5,2.2,0.54,1.25,1.94
23,2017-02-28,268287.000000,7.751910e+05,429829.000000,2493.577459,19325.115,51426.0,16844.0,8655.0,7174.0,...,120.828,253.983,127.022,186.601,209.641,6.9,2.6,0.65,1.26,1.93
24,2017-03-31,269261.000000,7.777100e+05,431475.000000,2499.800834,19325.115,53539.0,21342.0,8685.0,7404.0,...,123.301,254.398,130.424,186.875,207.121,7.2,2.7,0.66,1.14,1.83
25,2017-04-30,270457.000000,7.808330e+05,433431.000000,2512.312296,19325.115,56343.0,22134.0,9131.0,8902.0,...,123.262,254.000,133.045,187.884,204.318,7.2,2.5,0.79,1.13,1.64
26,2017-05-31,271538.000000,7.831900e+05,435102.000000,2528.656377,19325.115,59031.0,26088.0,9573.0,10260.0,...,123.860,253.582,130.086,186.345,208.244,7.3,2.5,0.90,1.01,1.49


In [36]:
df.isna().sum()

date                      0
ZABT                      0
ZATT                      0
ZSFH                      0
RSNA                      0
NYTPOP                    0
ACTLISCOU35620            0
NEWLISCOU35620            0
PENLISCOU35620            0
PRIREDCOU35620            0
NEWY636BPPRIV             0
SMU36356204200000001      0
SMU36356200500000001SA    0
SMU36356204300000001SA    0
ENUC356230010SA           0
NEWY636PCPI               0
SMU36356200500000003      0
SMU36356200500000002      0
SMU36356200500000011      0
LAUMT363562000000005      0
NEWY636URN                0
NEWY636NA                 0
NEWY636LFN                0
NEWY636NRMN               0
NEWY636PBSV               0
NEWY636FIRE               0
NEWY636MFG                0
NEWY636EDUH               0
NEWY636LEIH               0
CUURA101SA0               0
CUURA101SAF11             0
CUURA101SAF               0
CUURA101SA0E              0
CUURA101SEHF01            0
CUURA101SAT               0
CUURA101SAM         

In [37]:
df = df.set_index('date')

In [38]:
df.to_csv('df.csv')

### Notes

In [39]:
FRED
### https://fred.stlouisfed.org/categories/29191
GIS Data
###https://gis.ny.gov/gisdata/inventories/details.cfm?DSID=927
All Data Download Link
###https://data.nasdaq.com/tables/ZILLOW-DATA/export?api_key=es4ABcXZ_usFTxNf4mph
Zillow Research
### https://www.zillow.com/research/data/
Quandl Intro
### https://www.youtube.com/watch?v=AYwGunGBU40&ab_channel=AnalyticsAriel
-- Deal with Zips
### https://towardsdatascience.com/visualizing-data-at-the-zip-code-level-with-folium-d07ac983db20

SyntaxError: invalid syntax (<ipython-input-39-4777ebad8790>, line 3)